In [1]:
import os, sys, random
from math import floor
from operator import itemgetter

import shutil
import glob
from pathlib import Path

import pandas as pd
import numpy as np
import nibabel as nib

from sklearn.model_selection import StratifiedKFold
#from skimage.transform import resize
import h5py

import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive
from collections import defaultdict
from config import *
from images import *
from data_utils import *
from generator import *
from u_net_model import *
from reconstruct import *
from collections import defaultdict

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv3D, Dropout, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, PReLU, Conv3DTranspose, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
#tf.enable_eager_execution()

# force channels-last ordering
tf.keras.backend.set_image_data_format('channels_last')
print(tf.keras.backend.image_data_format())



channels_last


In [4]:
train_df=pd.read_csv(options["train_csv_path"])
train_df

,root_path,patient_id,study,mask1,mask2,flair,t2,pd,mprage,fold,f5_path
0,/media/marwa/F2F25460F2542ADD/MedicalAnalysis/...,training01,_01,mask1.nii,mask2.nii,flair.nii,t2.nii,pd.nii,mprage.nii,0.0,/media/marwa/F2F25460F2542ADD/MedicalAnalysis/...
1,/media/marwa/F2F25460F2542ADD/MedicalAnalysis/...,training01,_02,mask1.nii,mask2.nii,flair.nii,t2.nii,pd.nii,mprage.nii,2.0,/media/marwa/F2F25460F2542ADD/MedicalAnalysis/...
2,/media/marwa/F2F25460F2542ADD/MedicalAnalysis/...,training01,_03,mask1.nii,mask2.nii,flair.nii,t2.nii,pd.nii,mprage.nii,3.0,/media/marwa/F2F25460F2542ADD/MedicalAnalysis/...
3,/media/marwa/F2F25460F2542ADD/MedicalAnalysis/...,training01,_04,mask1.nii,mask2.nii,flair.nii,t2.nii,pd.nii,mprage.nii,1.0,/media/marwa/F2F25460F2542ADD/MedicalAnalysis/...
4,/media/marwa/F2F25460F2542ADD/MedicalAnalysis/...,training02,_01,mask1.nii,mask2.nii,flair.nii,t2.nii,pd.nii,mprage.nii,0.0,/media/marwa/F2F25460F2542ADD/MedicalAnalysis/...
5,/media/marwa/F2F25460F2542ADD/MedicalAnalysis/...,training02,_02,mask1.nii,mask2.nii,flair.nii,t2.nii,pd.nii,mprage.nii,1.0,/media/marwa/F2F25460F2542ADD/MedicalAnalysis/...
6,/media/marwa/F2F25460F2542ADD/MedicalAnalysis/...,training02,_03,mask1.nii,mask2.nii,flair.nii,t2.nii,pd.nii,mprage.nii,3.0,/media/marwa/F2F25460F2542ADD/MedicalAnalysis/...
7,/media/marwa/F2F25460F2542ADD/MedicalAnalysis/...,training02,_04,mask1.nii,mask2.nii,flair.nii,t2.nii,pd.nii,mprage.nii,2.0,/media/marwa/F2F25460F2542ADD/MedicalAnalysis/...
8,/media/marwa/F2F25460F2542ADD/MedicalAnalysis/...,training03,_01,mask1.nii,mask2.nii,flair.nii,t2.nii,pd.nii,mprage.nii,0.0,/media/marwa/F2F25460F2542ADD/MedicalAnalysis/...
9,/media/marwa/F2F25460F2542ADD/MedicalAnalysis/...,training03,_02,mask1.nii,mask2.nii,flair.nii,t2.nii,pd.nii,mprage.nii,0.0,/media/marwa/F2F25460F2542ADD/MedicalAnalysis/...


In [ ]:
data,ref,patches=load_data_patches(options)

get_data_path


/media/marwa/F2F25460F2542ADD/MedicalAnalysis/Code/ISBI_MS_Segmentation/data_utils.py:277: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  patches = [new_image[idx] for idx in slices]


(34912, 16, 16, 16, 4) (34912, 16, 16, 16, 1)
(34912, 16, 16, 16, 4) patches 34912 modalities 4
(29962, 16, 16, 16, 4) (29962, 16, 16, 16, 1)
(29962, 16, 16, 16, 4) patches 29962 modalities 4


In [ ]:
batch_size=options['batch_size']

In [ ]:
batch_size, patches

In [ ]:
gen=ISBIDataset(data=data, options=options, patches=patches)


In [ ]:
count=0
idx=0
for x,y in gen.__getitem__():
    print(x.shape,y.shape)

In [ ]:
gen.__len__()

In [ ]:
model=

In [2]:
# create model
model=U_Net_Model(options['input_shape'],
                      options['out_channels'],
                      depth = options['depth'], 
                      initial_learning_rate = options['initial_learning_rate'],
                      deconvolution = options['deconvolution'])
model.build((None,*options['input_shape']))
model.summary()
model.layers



input shape (None, 24, 24, 16, 4)
Model: "u__net__model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 24, 24, 16, 1)     4726966   
Total params: 4,726,966
Trainable params: 4,726,966
Non-trainable params: 0
_________________________________________________________________


In [20]:
files

array(['/media/marwa/F2F25460F2542ADD/MedicalAnalysis/DataSets/ISBI/h5df_files/file_training01_02.hdf5',
       '/media/marwa/F2F25460F2542ADD/MedicalAnalysis/DataSets/ISBI/h5df_files/file_training01_03.hdf5',
       '/media/marwa/F2F25460F2542ADD/MedicalAnalysis/DataSets/ISBI/h5df_files/file_training01_04.hdf5',
       '/media/marwa/F2F25460F2542ADD/MedicalAnalysis/DataSets/ISBI/h5df_files/file_training02_02.hdf5',
       '/media/marwa/F2F25460F2542ADD/MedicalAnalysis/DataSets/ISBI/h5df_files/file_training02_03.hdf5',
       '/media/marwa/F2F25460F2542ADD/MedicalAnalysis/DataSets/ISBI/h5df_files/file_training02_04.hdf5',
       '/media/marwa/F2F25460F2542ADD/MedicalAnalysis/DataSets/ISBI/h5df_files/file_training03_03.hdf5',
       '/media/marwa/F2F25460F2542ADD/MedicalAnalysis/DataSets/ISBI/h5df_files/file_training03_04.hdf5',
       '/media/marwa/F2F25460F2542ADD/MedicalAnalysis/DataSets/ISBI/h5df_files/file_training03_05.hdf5',
       '/media/marwa/F2F25460F2542ADD/MedicalAnalysis/D

In [45]:
(100)%(16+8)

4

In [40]:
(112/16+8)

15.0

# Reconstruct

In [4]:
train_df.columns

Index(['root_path', 'patient_id', 'study', 'mask1', 'mask2', 'flair', 't2',
       'pd', 'mprage', 'fold', 'f5_path'],
      dtype='object')

In [5]:
train_df = df.loc[df['fold'] != fold].reset_index(drop=True)
val_df = df.loc[df['fold'] == fold].reset_index(drop=True)

In [56]:
step_size=(16,16,8)
patch_size=options['patch_size']
orig_shape=(181, 217, 181)
for i,row in val_df.iterrows():
    patient_id,study,root_path = row['patient_id'],row['study'],row['root_path']
    # load all modalities
    images = []
    for modality in modalities:
        img_path = root_path+modality+".nii"
        img = load_img(img_path)

        img = normalize_image(img,datatype=np.float32)
        #padding=[(0,o_s%(p_s+s_s)) for o_s, p_s, s_s in zip(orig_shape, patch_size, step_size)]
        
        #img=np.pad(img, padding, 'constant')
        #print('padding',padding,img.shape)
        #print(img.shape)
        images.append(img)
    
    mask_path=root_path+"mask1.nii"
    img = np.moveaxis( np.stack(images),0,-1)
    
    mask = load_img(mask_path)
    
    #mask=np.pad(mask, padding, 'constant')
    shape=mask.shape

    image_real  = Reconstruct(i, shape, patch_size, False)
    predict  = Reconstruct(i, shape, patch_size, False)
    truth = Reconstruct(i, shape, patch_size, False)
    x=0
    while(x+patch_size[0] <= shape[0]):
        y=0
        while(y+patch_size[1]<= shape[1]):
            z=0
            while(z+patch_size[2] <= shape[2]):
                print(z, (shape[2]-(shape[2]%patch_size[2])))
                patch_start=(x,y,z)
                patch_end=tuple([p+s for p,s in zip(patch_start,patch_size)])
                #print('start',patch_start,'end',patch_end)
                patch_img=img[patch_start[0]:patch_end[0], patch_start[1]:patch_end[1], patch_start[2]:patch_end[2]]
                
                patch_mask=mask[patch_start[0]:patch_end[0], patch_start[1]:patch_end[1], patch_start[2]:patch_end[2]]
                #print('start',patch_start,'end',patch_end,'patch',patch_img.shape)
                #print(patch_img.shape,patch_mask.shape,result.shape,patch_index_row)
                image_real.add(patch_img[:,:,:,0], patch_start)
                #truth.add(patch_mask, patch_start)
        
                z+=step_size[2]
                
            y+=step_size[1]
        x+=step_size[0]
    dir_name = './model/h5df_data/recon/'
    os.makedirs(os.path.dirname(dir_name), exist_ok=True)
    file_name = dir_name+ patient_id+"_"+study
    image_real.store(file_name + "_image_real")
    truth.store(file_name + "_target")
    print('reconstruct',patient_id+"_"+study)
    break

(181, 217, 181)
(181, 217, 181)
(181, 217, 181)
(181, 217, 181)
0 176
start (0, 0, 0) end (24, 24, 16) patch (24, 24, 16, 4)
8 176
start (0, 0, 8) end (24, 24, 24) patch (24, 24, 16, 4)
16 176
start (0, 0, 16) end (24, 24, 32) patch (24, 24, 16, 4)
24 176
start (0, 0, 24) end (24, 24, 40) patch (24, 24, 16, 4)
32 176
start (0, 0, 32) end (24, 24, 48) patch (24, 24, 16, 4)
40 176
start (0, 0, 40) end (24, 24, 56) patch (24, 24, 16, 4)
48 176
start (0, 0, 48) end (24, 24, 64) patch (24, 24, 16, 4)
56 176
start (0, 0, 56) end (24, 24, 72) patch (24, 24, 16, 4)
64 176
start (0, 0, 64) end (24, 24, 80) patch (24, 24, 16, 4)
72 176
start (0, 0, 72) end (24, 24, 88) patch (24, 24, 16, 4)
80 176
start (0, 0, 80) end (24, 24, 96) patch (24, 24, 16, 4)
88 176
start (0, 0, 88) end (24, 24, 104) patch (24, 24, 16, 4)
96 176
start (0, 0, 96) end (24, 24, 112) patch (24, 24, 16, 4)
104 176
start (0, 0, 104) end (24, 24, 120) patch (24, 24, 16, 4)
112 176
start (0, 0, 112) end (24, 24, 128) patch (24

8 176
start (0, 96, 8) end (24, 120, 24) patch (24, 24, 16, 4)
16 176
start (0, 96, 16) end (24, 120, 32) patch (24, 24, 16, 4)
24 176
start (0, 96, 24) end (24, 120, 40) patch (24, 24, 16, 4)
32 176
start (0, 96, 32) end (24, 120, 48) patch (24, 24, 16, 4)
40 176
start (0, 96, 40) end (24, 120, 56) patch (24, 24, 16, 4)
48 176
start (0, 96, 48) end (24, 120, 64) patch (24, 24, 16, 4)
56 176
start (0, 96, 56) end (24, 120, 72) patch (24, 24, 16, 4)
64 176
start (0, 96, 64) end (24, 120, 80) patch (24, 24, 16, 4)
72 176
start (0, 96, 72) end (24, 120, 88) patch (24, 24, 16, 4)
80 176
start (0, 96, 80) end (24, 120, 96) patch (24, 24, 16, 4)
88 176
start (0, 96, 88) end (24, 120, 104) patch (24, 24, 16, 4)
96 176
start (0, 96, 96) end (24, 120, 112) patch (24, 24, 16, 4)
104 176
start (0, 96, 104) end (24, 120, 120) patch (24, 24, 16, 4)
112 176
start (0, 96, 112) end (24, 120, 128) patch (24, 24, 16, 4)
120 176
start (0, 96, 120) end (24, 120, 136) patch (24, 24, 16, 4)
128 176
start (0

160 176
start (0, 176, 160) end (24, 200, 176) patch (24, 24, 16, 4)
0 176
start (0, 192, 0) end (24, 216, 16) patch (24, 24, 16, 4)
8 176
start (0, 192, 8) end (24, 216, 24) patch (24, 24, 16, 4)
16 176
start (0, 192, 16) end (24, 216, 32) patch (24, 24, 16, 4)
24 176
start (0, 192, 24) end (24, 216, 40) patch (24, 24, 16, 4)
32 176
start (0, 192, 32) end (24, 216, 48) patch (24, 24, 16, 4)
40 176
start (0, 192, 40) end (24, 216, 56) patch (24, 24, 16, 4)
48 176
start (0, 192, 48) end (24, 216, 64) patch (24, 24, 16, 4)
56 176
start (0, 192, 56) end (24, 216, 72) patch (24, 24, 16, 4)
64 176
start (0, 192, 64) end (24, 216, 80) patch (24, 24, 16, 4)
72 176
start (0, 192, 72) end (24, 216, 88) patch (24, 24, 16, 4)
80 176
start (0, 192, 80) end (24, 216, 96) patch (24, 24, 16, 4)
88 176
start (0, 192, 88) end (24, 216, 104) patch (24, 24, 16, 4)
96 176
start (0, 192, 96) end (24, 216, 112) patch (24, 24, 16, 4)
104 176
start (0, 192, 104) end (24, 216, 120) patch (24, 24, 16, 4)
112 17

8 176
start (16, 80, 8) end (40, 104, 24) patch (24, 24, 16, 4)
16 176
start (16, 80, 16) end (40, 104, 32) patch (24, 24, 16, 4)
24 176
start (16, 80, 24) end (40, 104, 40) patch (24, 24, 16, 4)
32 176
start (16, 80, 32) end (40, 104, 48) patch (24, 24, 16, 4)
40 176
start (16, 80, 40) end (40, 104, 56) patch (24, 24, 16, 4)
48 176
start (16, 80, 48) end (40, 104, 64) patch (24, 24, 16, 4)
56 176
start (16, 80, 56) end (40, 104, 72) patch (24, 24, 16, 4)
64 176
start (16, 80, 64) end (40, 104, 80) patch (24, 24, 16, 4)
72 176
start (16, 80, 72) end (40, 104, 88) patch (24, 24, 16, 4)
80 176
start (16, 80, 80) end (40, 104, 96) patch (24, 24, 16, 4)
88 176
start (16, 80, 88) end (40, 104, 104) patch (24, 24, 16, 4)
96 176
start (16, 80, 96) end (40, 104, 112) patch (24, 24, 16, 4)
104 176
start (16, 80, 104) end (40, 104, 120) patch (24, 24, 16, 4)
112 176
start (16, 80, 112) end (40, 104, 128) patch (24, 24, 16, 4)
120 176
start (16, 80, 120) end (40, 104, 136) patch (24, 24, 16, 4)
1

160 176
start (16, 160, 160) end (40, 184, 176) patch (24, 24, 16, 4)
0 176
start (16, 176, 0) end (40, 200, 16) patch (24, 24, 16, 4)
8 176
start (16, 176, 8) end (40, 200, 24) patch (24, 24, 16, 4)
16 176
start (16, 176, 16) end (40, 200, 32) patch (24, 24, 16, 4)
24 176
start (16, 176, 24) end (40, 200, 40) patch (24, 24, 16, 4)
32 176
start (16, 176, 32) end (40, 200, 48) patch (24, 24, 16, 4)
40 176
start (16, 176, 40) end (40, 200, 56) patch (24, 24, 16, 4)
48 176
start (16, 176, 48) end (40, 200, 64) patch (24, 24, 16, 4)
56 176
start (16, 176, 56) end (40, 200, 72) patch (24, 24, 16, 4)
64 176
start (16, 176, 64) end (40, 200, 80) patch (24, 24, 16, 4)
72 176
start (16, 176, 72) end (40, 200, 88) patch (24, 24, 16, 4)
80 176
start (16, 176, 80) end (40, 200, 96) patch (24, 24, 16, 4)
88 176
start (16, 176, 88) end (40, 200, 104) patch (24, 24, 16, 4)
96 176
start (16, 176, 96) end (40, 200, 112) patch (24, 24, 16, 4)
104 176
start (16, 176, 104) end (40, 200, 120) patch (24, 24

...

144 176
start (128, 32, 144) end (152, 56, 160) patch (24, 24, 16, 4)
152 176
start (128, 32, 152) end (152, 56, 168) patch (24, 24, 16, 4)
160 176
start (128, 32, 160) end (152, 56, 176) patch (24, 24, 16, 4)
0 176
start (128, 48, 0) end (152, 72, 16) patch (24, 24, 16, 4)
8 176
start (128, 48, 8) end (152, 72, 24) patch (24, 24, 16, 4)
16 176
start (128, 48, 16) end (152, 72, 32) patch (24, 24, 16, 4)
24 176
start (128, 48, 24) end (152, 72, 40) patch (24, 24, 16, 4)
32 176
start (128, 48, 32) end (152, 72, 48) patch (24, 24, 16, 4)
40 176
start (128, 48, 40) end (152, 72, 56) patch (24, 24, 16, 4)
48 176
start (128, 48, 48) end (152, 72, 64) patch (24, 24, 16, 4)
56 176
start (128, 48, 56) end (152, 72, 72) patch (24, 24, 16, 4)
64 176
start (128, 48, 64) end (152, 72, 80) patch (24, 24, 16, 4)
72 176
start (128, 48, 72) end (152, 72, 88) patch (24, 24, 16, 4)
80 176
start (128, 48, 80) end (152, 72, 96) patch (24, 24, 16, 4)
88 176
start (128, 48, 88) end (152, 72, 104) patch (24, 

88 176
start (128, 128, 88) end (152, 152, 104) patch (24, 24, 16, 4)
96 176
start (128, 128, 96) end (152, 152, 112) patch (24, 24, 16, 4)
104 176
start (128, 128, 104) end (152, 152, 120) patch (24, 24, 16, 4)
112 176
start (128, 128, 112) end (152, 152, 128) patch (24, 24, 16, 4)
120 176
start (128, 128, 120) end (152, 152, 136) patch (24, 24, 16, 4)
128 176
start (128, 128, 128) end (152, 152, 144) patch (24, 24, 16, 4)
136 176
start (128, 128, 136) end (152, 152, 152) patch (24, 24, 16, 4)
144 176
start (128, 128, 144) end (152, 152, 160) patch (24, 24, 16, 4)
152 176
start (128, 128, 152) end (152, 152, 168) patch (24, 24, 16, 4)
160 176
start (128, 128, 160) end (152, 152, 176) patch (24, 24, 16, 4)
0 176
start (128, 144, 0) end (152, 168, 16) patch (24, 24, 16, 4)
8 176
start (128, 144, 8) end (152, 168, 24) patch (24, 24, 16, 4)
16 176
start (128, 144, 16) end (152, 168, 32) patch (24, 24, 16, 4)
24 176
start (128, 144, 24) end (152, 168, 40) patch (24, 24, 16, 4)
32 176
start

24 176
start (144, 16, 24) end (168, 40, 40) patch (24, 24, 16, 4)
32 176
start (144, 16, 32) end (168, 40, 48) patch (24, 24, 16, 4)
40 176
start (144, 16, 40) end (168, 40, 56) patch (24, 24, 16, 4)
48 176
start (144, 16, 48) end (168, 40, 64) patch (24, 24, 16, 4)
56 176
start (144, 16, 56) end (168, 40, 72) patch (24, 24, 16, 4)
64 176
start (144, 16, 64) end (168, 40, 80) patch (24, 24, 16, 4)
72 176
start (144, 16, 72) end (168, 40, 88) patch (24, 24, 16, 4)
80 176
start (144, 16, 80) end (168, 40, 96) patch (24, 24, 16, 4)
88 176
start (144, 16, 88) end (168, 40, 104) patch (24, 24, 16, 4)
96 176
start (144, 16, 96) end (168, 40, 112) patch (24, 24, 16, 4)
104 176
start (144, 16, 104) end (168, 40, 120) patch (24, 24, 16, 4)
112 176
start (144, 16, 112) end (168, 40, 128) patch (24, 24, 16, 4)
120 176
start (144, 16, 120) end (168, 40, 136) patch (24, 24, 16, 4)
128 176
start (144, 16, 128) end (168, 40, 144) patch (24, 24, 16, 4)
136 176
start (144, 16, 136) end (168, 40, 152) 

8 176
start (144, 112, 8) end (168, 136, 24) patch (24, 24, 16, 4)
16 176
start (144, 112, 16) end (168, 136, 32) patch (24, 24, 16, 4)
24 176
start (144, 112, 24) end (168, 136, 40) patch (24, 24, 16, 4)
32 176
start (144, 112, 32) end (168, 136, 48) patch (24, 24, 16, 4)
40 176
start (144, 112, 40) end (168, 136, 56) patch (24, 24, 16, 4)
48 176
start (144, 112, 48) end (168, 136, 64) patch (24, 24, 16, 4)
56 176
start (144, 112, 56) end (168, 136, 72) patch (24, 24, 16, 4)
64 176
start (144, 112, 64) end (168, 136, 80) patch (24, 24, 16, 4)
72 176
start (144, 112, 72) end (168, 136, 88) patch (24, 24, 16, 4)
80 176
start (144, 112, 80) end (168, 136, 96) patch (24, 24, 16, 4)
88 176
start (144, 112, 88) end (168, 136, 104) patch (24, 24, 16, 4)
96 176
start (144, 112, 96) end (168, 136, 112) patch (24, 24, 16, 4)
104 176
start (144, 112, 104) end (168, 136, 120) patch (24, 24, 16, 4)
112 176
start (144, 112, 112) end (168, 136, 128) patch (24, 24, 16, 4)
120 176
start (144, 112, 120

120 176
start (144, 192, 120) end (168, 216, 136) patch (24, 24, 16, 4)
128 176
start (144, 192, 128) end (168, 216, 144) patch (24, 24, 16, 4)
136 176
start (144, 192, 136) end (168, 216, 152) patch (24, 24, 16, 4)
144 176
start (144, 192, 144) end (168, 216, 160) patch (24, 24, 16, 4)
152 176
start (144, 192, 152) end (168, 216, 168) patch (24, 24, 16, 4)
160 176
start (144, 192, 160) end (168, 216, 176) patch (24, 24, 16, 4)
reconstruct training01__01


In [58]:
p='/media/marwa/F2F25460F2542ADD/MedicalAnalysis/Code/ISBI_MS_Segmentation/model/h5df_data/recon/training01__01_image_real.nii.gz'
img=load_img(p)
show3D_images([img])

interactive(children=(IntSlider(value=90, description='id', max=180), Output()), _dom_classes=('widget-interac…

In [ ]:
weight_path = './dice_coefficient_loss/fold_0_weights-01-0.48.hdf5',
           
fold=0
modalities = options['modalities']
df=pd.read_csv(options["train_csv_path"])
#df['path']=df['path'].str.replace('.', '../input/isbi-dataset-ms-lesion-segmentation',1)
#df['mask1_path']=df['mask1_path'].str.replace('.', '../input/isbi-dataset-ms-lesion-segmentation',1)


train_df = df.loc[df['fold'] != fold].reset_index(drop=True)
val_df = df.loc[df['fold'] == fold].reset_index(drop=True)

model=U_Net_Model(options['input_shape'],
                      options['out_channels'],
                      depth = options['depth'], 
                      initial_learning_rate = options['initial_learning_rate'],
                      deconvolution = options['deconvolution'])

model.build((None ,*config.input_shape))
model.load_weights('./dice_coefficient_loss/fold_0_weights-01-0.48.hdf5') 

for i,row in val_df.iterrows():
    patient_id,study,root_path = row['patient_id'],row['study'],row['root_path']
    # load all modalities
    images = []
    for modality in modalities:
        img_path = root_path+modality+".nii"
        img = load_img(img_path)

        img = normalize_image(img,datatype=np.float32)
        images.append(img)

    img = np.stack(images)
    mask = load_img(mask_path)

    shape=mask.shape
    step_size=(16,16,8)
    patch_size=options['patch_size']
    predict  = Reconstruct(i, shape, patch_size, False)
    truth = Reconstruct(i, shape, patch_size, False)
    
    x=0
    while(True):
        y=0
        while(True):
            z=0
            while(True):
                patch_start=(x,y,z)
                patch_end=(p+s for p,s in zip(patch_start,patch_size))
                patch_img=img[*[p:s for p,s in zip(patch_start,patch_size)]
                patch_mask=mask[*[p:s for p,s in zip(patch_start,patch_size)]

In [3]:
weight_path = './dice_coefficient_loss/fold_0_weights-01-0.48.hdf5',
           
fold=0
modalities = options['modalities']
df=pd.read_csv(options["train_csv_path"])
#df['path']=df['path'].str.replace('.', '../input/isbi-dataset-ms-lesion-segmentation',1)
#df['mask1_path']=df['mask1_path'].str.replace('.', '../input/isbi-dataset-ms-lesion-segmentation',1)


train_df = df.loc[df['fold'] != fold].reset_index(drop=True)
val_df = df.loc[df['fold'] == fold].reset_index(drop=True)

model=U_Net_Model(options['input_shape'],
                      options['out_channels'],
                      depth = options['depth'], 
                      initial_learning_rate = options['initial_learning_rate'],
                      deconvolution = options['deconvolution'])

model.build((None ,*options['input_shape']))
model.load_weights('./model_weights.hdf5') 


step_size=(16,16,8)
patch_size=options['patch_size']
orig_shape=(181, 217, 181)
for i,row in val_df.iterrows():
    patient_id,study,root_path = row['patient_id'],row['study'],row['root_path']
    # load all modalities
    images = []
    for modality in modalities:
        img_path = root_path+modality+".nii"
        img = load_img(img_path)

        img = normalize_image(img,datatype=np.float32)
        #padding=[(0,o_s%(p_s+s_s)) for o_s, p_s, s_s in zip(orig_shape, patch_size, step_size)]
        
        #img=np.pad(img, padding, 'constant')
        #print('padding',padding,img.shape)
        #print(img.shape)
        images.append(img)
        
    
    mask_path=root_path+"mask1.nii"
    img = np.moveaxis( np.stack(images),0,-1)
    
    mask = load_img(mask_path)
    
    #mask=np.pad(mask, padding, 'constant')
    shape=mask.shape

    image_real  = Reconstruct(i, shape, patch_size, False)
    predict  = Reconstruct(i, shape, patch_size, False)
    truth = Reconstruct(i, shape, patch_size, False)
    x=0
    while(x+patch_size[0] <= shape[0]):
        y=0
        while(y+patch_size[1]<= shape[1]):
            z=0
            while(z+patch_size[2] <= shape[2]):
                #print(z, (shape[2]-(shape[2]%patch_size[2])))
                patch_start=(x,y,z)
                patch_end=tuple([p+s for p,s in zip(patch_start,patch_size)])
                #print('start',patch_start,'end',patch_end)
                patch_img=img[patch_start[0]:patch_end[0], patch_start[1]:patch_end[1], patch_start[2]:patch_end[2]]
                
                patch_mask=mask[patch_start[0]:patch_end[0], patch_start[1]:patch_end[1], patch_start[2]:patch_end[2]]
                #print('start',patch_start,'end',patch_end,'patch',patch_img.shape)
                #print(patch_img.shape,patch_mask.shape,result.shape,patch_index_row)
                image_real.add(patch_img[:,:,:,0], patch_start)
                truth.add(patch_mask, patch_start)
                predict_img=model.predict(np.expand_dims(patch_img,axis=0))
                
                predict.add(np.squeeze(np.squeeze(predict_img, axis=0),axis=-1), patch_start)
        
                z+=step_size[2]
                
            y+=step_size[1]
        x+=step_size[0]
    dir_name = './model/h5df_data/recon/'
    os.makedirs(os.path.dirname(dir_name), exist_ok=True)
    file_name = dir_name+ patient_id+"_"+study
    image_real.store(file_name + "_image_real")
    truth.store(file_name + "_target")
    predict.store(file_name + "_predict")
    print('reconstruct',patient_id+"_"+study)
    break

input shape (None, 16, 16, 16, 4)
0 176
8 176
16 176
24 176
32 176
40 176
48 176
56 176
64 176
72 176
80 176
88 176
96 176
104 176
112 176
120 176
128 176
136 176
144 176
152 176
160 176
0 176
8 176
16 176
24 176
32 176
40 176
48 176
56 176
64 176
72 176
80 176
88 176
96 176
104 176
112 176
120 176
128 176
136 176
144 176
152 176
160 176
0 176
8 176
16 176
24 176
32 176
40 176
48 176
56 176
64 176
72 176
80 176
88 176
96 176
104 176
112 176
120 176
128 176
136 176
144 176
152 176
160 176
0 176
8 176
16 176
24 176
32 176
40 176
48 176
56 176
64 176
72 176
80 176
88 176
96 176
104 176
112 176
120 176
128 176
136 176
144 176
152 176
160 176
0 176
8 176
16 176
24 176
32 176
40 176
48 176
56 176
64 176
72 176
80 176
88 176
96 176
104 176
112 176
120 176
128 176
136 176
144 176
152 176
160 176
0 176
8 176
16 176
24 176
32 176
40 176
48 176
56 176
64 176
72 176
80 176
88 176
96 176
104 176
112 176
120 176
128 176
136 176
144 176
152 176
160 176
0 176
8 176
16 176
24 176
32 176
40 176
48 176
5

64 176
72 176
80 176
88 176
96 176
104 176
112 176
120 176
128 176
136 176
144 176
152 176
160 176
0 176
8 176
16 176
24 176
32 176
40 176
48 176
56 176
64 176
72 176
80 176
88 176
96 176
104 176
112 176
120 176
128 176
136 176
144 176
152 176
160 176
0 176
8 176
16 176
24 176
32 176
40 176
48 176
56 176
64 176
72 176
80 176
88 176
96 176
104 176
112 176
120 176
128 176
136 176
144 176
152 176
160 176
0 176
8 176
16 176
24 176
32 176
40 176
48 176
56 176
64 176
72 176
80 176
88 176
96 176
104 176
112 176
120 176
128 176
136 176
144 176
152 176
160 176
0 176
8 176
16 176
24 176
32 176
40 176
48 176
56 176
64 176
72 176
80 176
88 176
96 176
104 176
112 176
120 176
128 176
136 176
144 176
152 176
160 176
0 176
8 176
16 176
24 176
32 176
40 176
48 176
56 176
64 176
72 176
80 176
88 176
96 176
104 176
112 176
120 176
128 176
136 176
144 176
152 176
160 176
0 176
8 176
16 176
24 176
32 176
40 176
48 176
56 176
64 176
72 176
80 176
88 176
96 176
104 176
112 176
120 176
128 176
136 176
144 176

160 176
0 176
8 176
16 176
24 176
32 176
40 176
48 176
56 176
64 176
72 176
80 176
88 176
96 176
104 176
112 176
120 176
128 176
136 176
144 176
152 176
160 176
0 176
8 176
16 176
24 176
32 176
40 176
48 176
56 176
64 176
72 176
80 176
88 176
96 176
104 176
112 176
120 176
128 176
136 176
144 176
152 176
160 176
0 176
8 176
16 176
24 176
32 176
40 176
48 176
56 176
64 176
72 176
80 176
88 176
96 176
104 176
112 176
120 176
128 176
136 176
144 176
152 176
160 176
0 176
8 176
16 176
24 176
32 176
40 176
48 176
56 176
64 176
72 176
80 176
88 176
96 176
104 176
112 176
120 176
128 176
136 176
144 176
152 176
160 176
0 176
8 176
16 176
24 176
32 176
40 176
48 176
56 176
64 176
72 176
80 176
88 176
96 176
104 176
112 176
120 176
128 176
136 176
144 176
152 176
160 176
0 176
8 176
16 176
24 176
32 176
40 176
48 176
56 176
64 176
72 176
80 176
88 176
96 176
104 176
112 176
120 176
128 176
136 176
144 176
152 176
160 176
0 176
8 176
16 176
24 176
32 176
40 176
48 176
56 176
64 176
72 176
80 176

In [4]:
img=load_img('./model/h5df_data/recon/training01__01_image_real.nii.gz')
target=load_img('./model/h5df_data/recon/training01__01_target.nii.gz')
predict=load_img('./model/h5df_data/recon/training01__01_predict.nii.gz')
show3D_images([img,target,predict])


interactive(children=(IntSlider(value=90, description='id', max=180), Output()), _dom_classes=('widget-interac…

(181, 217, 181)

In [13]:
img1=np.moveaxis(img,(0,1,2),(2,1,1))
img.shape, img1.shape
show3D_images([img1])

interactive(children=(IntSlider(value=90, description='id', max=180), Output()), _dom_classes=('widget-interac…

In [6]:
from metric import*

In [9]:
dice_coefficient(np.expand_dims(np.expand_dims(target,axis=0),-1),np.expand_dims(np.expand_dims(predict,axis=0),axis=-1))

<tf.Tensor: shape=(), dtype=float64, numpy=0.4350510980648213>

# Model train

In [ ]:
df = pd.read_csv(options['train_csv_path'])
losses={"BCEDiceLoss":BCEDiceLoss}


for k in losses.keys():
    for fold in range(options['k_fold']):
        options['weights_file_path'] = "./"+k+"/"
        os.makedirs(options['weights_file_path'], exist_ok=True)

        train_files,train_files_ref,train_patches=load_data_patches(options, phase='train',fold=fold)
        valid_files,valid_files_ref,valid_patches=load_data_patches(options, phase='valid',fold=fold)
        
        train_generator=ISBIDataset(data=train_files, options=options, patches=train_patches)
        valid_generator=ISBIDataset(data=valid_files, options=options, patches=valid_patches)
        
        
        print ('-'*100)
        print ("Fold:", fold)

        # create model
        model=U_Net_Model(options['input_shape'],
                              options['out_channels'],
                              depth = options['depth'], 
                              initial_learning_rate = options['initial_learning_rate'],
                              deconvolution = options['deconvolution'])
        #model.summary()
        if not isinstance(options['metrics'], list):
            options['metrics'] = [options['metrics']]
        model.compile(optimizer=Adam(lr=options['initial_learning_rate']), 
                      loss=losses[k], metrics=options['metrics'],run_eagerly=True)
        callbacks = model.get_callbacks(options['weights_file_path'], str(fold),
                                initial_learning_rate=options['initial_learning_rate'],
                                learning_rate_drop=options['learning_rate_drop'],
                                learning_rate_patience=options['patience'],
                                early_stopping_patience=options['early_stop'])
        

        epochs = options['n_epochs']
        
        # Instantiate an optimizer.
        optimizer = keras.optimizers.Adam(options['initial_learning_rate'])

        # Prepare the metrics.
        train_acc_metric = DiceCoefficient()
        val_acc_metric = DiceCoefficient()
        import time
        for epoch in range(epochs):
            print("\nStart of epoch %d" % (epoch,))
            start_time = time.time()
            # Iterate over the batches of the dataset.
            for step, (x_batch_train, y_batch_train) in enumerate(train_generator.__getitem__()):
                x_batch_train.shape
                # Open a GradientTape to record the operations run
                # during the forward pass, which enables auto-differentiation.
                with tf.GradientTape() as tape:

                    # Run the forward pass of the layer.
                    # The operations that the layer applies
                    # to its inputs are going to be recorded
                    # on the GradientTape.
                    logits = model(x_batch_train, training=True)  # Logits for this minibatch

                    # Compute the loss value for this minibatch.
                    loss_value = BCEDiceLoss(y_batch_train, logits)

                # Use the gradient tape to automatically retrieve
                # the gradients of the trainable variables with respect to the loss.
                grads = tape.gradient(loss_value, model.trainable_weights)

                # Run one step of gradient descent by updating
                # the value of the variables to minimize the loss.
                optimizer.apply_gradients(zip(grads, model.trainable_weights))
                
                # Update training metric.
                train_acc_metric.update_state(y_batch_train, logits)

                # Log every 200 batches.
                if step % 200 == 0:
                    print(
                        "Training loss (for one batch) at step %d: %.4f"
                        % (step, float(loss_value))
                    )
                    print("Seen so far: %s samples" % ((step + 1) * 64))
            # Display metrics at the end of each epoch.
            train_acc = train_acc_metric.result()
            print("Training acc over epoch: %.4f" % (float(train_acc),))

            # Reset training metrics at the end of each epoch
            train_acc_metric.reset_states()
            # Run a validation loop at the end of each epoch.
            for x_batch_val, y_batch_val in valid_generator.__getitem__():
                val_logits = model(x_batch_val, training=False)
                # Update val metrics
                val_acc_metric.update_state(y_batch_val, val_logits)
            val_acc = val_acc_metric.result()
            val_acc_metric.reset_states()
            print("Validation acc: %.4f" % (float(val_acc),))
            print("Time taken: %.2fs" % (time.time() - start_time))
        #save model weights
        model.save_weights('./model_weights.hdf5')
        break